In [ ]:
import requests
import csv
import pandas as pd

from datetime import datetime
from google.auth import default
from google.auth.transport.requests import Request

In [ ]:
def get_gcloud_credential():
    credentials, project = default()
    if not credentials.valid:
        if credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
    return credentials

In [ ]:
def circle_search(credentials, coordinate, radius):
    api_url = 'https://places.googleapis.com/v1/places:searchNearby'
    lat, long = [float(element) for element in coordinate.split(',')]
    field = ('places.displayName,'
             'places.types,'
             'places.priceLevel,'
             'places.rating')
    
    header = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {credentials.token}',
        'X-Goog-FieldMask': field,
        'X-Goog-User-Project': credentials.quota_project_id
    }
    
    payload = {
        'locationRestriction': {
            'circle': {
                'center': {
                    'latitude': lat,
                    'longitude': long
                },
                'radius': radius
            }
        }
    }
    
    response = requests.post(api_url, headers=header, json=payload)
    if response.status_code == 200:
        result = response.json()
        return result
    else:
        print(f'Error: {response.status_code}')
        print(response.json())
        return None

In [ ]:
def dataset_maker(data):
    now = datetime.now().strftime('%d-%m-%y_%H%M')
    banned_types = ['point_of_interest', 'establishment']
    file_path = f'dataset-output/export_{now}.csv'
    dataset = []
    
    for item in data.get('places', []):
        types = [t for t in item.get('types', []) if t not in banned_types]
        row = {
            'Name': item.get('displayName', {}).get('text', ),
            'Types': ', '.join(types),
            'Price Level': item.get('priceLevel', ),
            'Rating': item.get('rating', )
        }
        dataset.append(row)
    
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=dataset[0].keys())
        writer.writeheader()
        writer.writerows(dataset)
    
    print(f'Dataset saved to {file_path}')

In [ ]:
coordination = '-8.67039555301166, 115.21251859347734'
credential = get_gcloud_credential()
circle_radius = 500

places = circle_search(credential, coordination, circle_radius)

if places:
    for place in places.get('places', []):
        display_name = place.get('displayName', 'no displayname found')
        print(f'Name        : {display_name.get("text", "name not found")}')
        print(f'Types       : {place.get("types", "types not found")}')
        print(f'Price Level : {place.get("priceLevel", "no price level found")}')
        print(f'Rating      : {place.get("rating", "rating not found")}')
        print('---------------------------------------------------------')
else:
    print('No places found')

dataset_maker(places)

In [ ]:
coord = pd.read_csv('coordinates-list/hexagonal-packing-v1.csv')
radius = 500
credential = get_gcloud_credential()

for i in range(10):
    places = circle_search(credential, coord['coordinate'][i], radius)
    dataset_maker(places) 